In [ ]:
import json
import csv
from typing import List
import numpy as np
import torch
from torch.nn import Softmax, Sigmoid, BCELoss
import pandas
from datasets import load_from_disk
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BatchEncoding, PreTrainedModel, PretrainedConfig, BertModel, BertConfig, AdamW, BartForConditionalGeneration, BartTokenizer, GPT2Tokenizer, GPT2LMHeadModel
import sys
import tqdm

sys.path.append('..')
from tools.BasicUtils import ntopidx, SparseRetrieveSentForPairCoOccur

<h1> Training test

<h2> Score function 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_special_tokens({'additional_special_tokens' : ['<RELATION>']})

In [ ]:
# Define dataset and training arguement
dataset = load_from_disk('data/single-ollie')
training_args = TrainingArguments("data/single-ollie-sf1", evaluation_strategy="epoch")

In [ ]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Define data pre-process function and reform dataset
def preprocess_sf1(examples):
    # return BatchEncoding(tokenizer(examples['ent1'], examples['ent2'], padding=True, truncation=True, max_length=100, return_tensors="pt"))
    query = ['%s <RELATION> %s' % (ent1, ent2) for ent1, ent2 in zip(examples['ent1'], examples['ent2'])]
    return tokenizer(query, examples["sent"], padding=True, truncation=True, max_length=100)
    
train_dataset = dataset['train'].map(preprocess_sf1, batched=True)
valid_dataset = dataset['valid'].map(preprocess_sf1, batched=True)

In [ ]:
# Define trainer and do training
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=valid_dataset)
trainer.train()

<h2> Score function 2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_special_tokens({'additional_special_tokens' : ['<RELATION>']})

In [ ]:
# Define dataset and training arguement
dataset = load_from_disk('data/single-ollie')
training_args = TrainingArguments("data/single-ollie-sf2", evaluation_strategy="epoch")

In [ ]:
# Load model
class ScoreFunction2(PreTrainedModel):
    def __init__(self, config:PretrainedConfig):
        super().__init__(config)
        self._context_encoder = BertModel(config)
        self._query_encoder = BertModel(config)
        self._sigmoid = Sigmoid()

    def forward(self, 
        context_input_ids,
        query_input_ids,
        context_token_type_ids=None,
        context_attention_mask=None,
        query_token_type_ids=None,
        query_attention_mask=None):
        context_inputs = {'input_ids': context_input_ids, 'token_type_ids': context_token_type_ids, 'attention_mask': context_attention_mask}
        query_inputs = {'input_ids': query_input_ids, 'token_type_ids': query_token_type_ids, 'attention_mask': query_attention_mask}
        context_emb = self._context_encoder(**context_inputs).last_hidden_state[:, 0, :]
        query_emb = self._query_encoder(**query_inputs).last_hidden_state[:, 0, :]
        score = self._sigmoid(torch.mul(context_emb, query_emb).sum(dim=1))
        return score

model = ScoreFunction2(BertConfig())
model._query_encoder.resize_token_embeddings(len(tokenizer))

In [ ]:
def preprocess_sf2(examples):
    query = ['%s <RELATION> %s' % (ent1, ent2) for ent1, ent2 in zip(examples['ent1'], examples['ent2'])]
    context_tokenized = tokenizer(examples["sent"], padding=True, truncation=True, max_length=100)
    query_tokenized = tokenizer(query, padding=True, truncation=True, max_length=100)
    return {'context_input_ids': context_tokenized['input_ids'], 
            'context_token_type_ids': context_tokenized['token_type_ids'], 
            'context_attention_mask': context_tokenized['attention_mask'],
            'query_input_ids': query_tokenized['input_ids'], 
            'query_token_type_ids': query_tokenized['token_type_ids'], 
            'query_attention_mask': query_tokenized['attention_mask']}

train_dataset = dataset['train'].map(preprocess_sf2, batched=True)
valid_dataset = dataset['valid'].map(preprocess_sf2, batched=True)

In [ ]:
class ScoreFunction2Trainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss_function = BCELoss()
        loss = loss_function(outputs, labels)
        return (loss, outputs) if return_outputs else loss
        
trainer = ScoreFunction2Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=valid_dataset)
trainer.train()

<h2> Score function 3

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
tokenizer.add_special_tokens({'additional_special_tokens' : ['<ENT1>', '<ENT2>', '<RELATION>']})

In [ ]:
# Define dataset and training arguement
dataset = load_from_disk('data/single-ollie-pos-only')
training_args_1 = TrainingArguments("data/single-ollie-sf3", evaluation_strategy="epoch")
training_args_2 = TrainingArguments("data/single-ollie-sf3_2", evaluation_strategy="epoch")

In [ ]:
# Load model
relation_extractor = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
relation_extractor.resize_token_embeddings(len(tokenizer))

In [ ]:
reconstructor = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
reconstructor.resize_token_embeddings(len(tokenizer))

In [ ]:
def preprocess_sf3_1(examples):
    conditions = ['%s <ENT1> %s <ENT2> %s' % (sent, ent1, ent2) for ent1, ent2, sent in zip(examples['ent1'], examples['ent2'], examples['sent'])]
    rel_tokenized = tokenizer(examples["sent"], padding='max_length', truncation=True, max_length=100)
    condition_tokenized = tokenizer(conditions, padding='max_length', truncation=True, max_length=100)
    return {'input_ids': condition_tokenized['input_ids'],
            'labels' : rel_tokenized['input_ids']}

def preprocess_sf3_2(examples):
    conditions = ['<ENT1> %s <ENT2> %s <RELATION> %s' % (ent1, ent2, rel) for ent1, ent2, rel in zip(examples['ent1'], examples['ent2'], examples['rel'])]
    sent_tokenized = tokenizer(examples["sent"], padding='max_length', truncation=True, max_length=100)
    condition_tokenized = tokenizer(conditions, padding='max_length', truncation=True, max_length=100)
    return {'input_ids': condition_tokenized['input_ids'],
            'labels' : sent_tokenized['input_ids']}

In [ ]:
train_dataset_1 = dataset['train'].map(preprocess_sf3_1, batched=True)
valid_dataset_1 = dataset['valid'].map(preprocess_sf3_1, batched=True)
train_dataset_2 = dataset['train'].map(preprocess_sf3_2, batched=True)
valid_dataset_2 = dataset['valid'].map(preprocess_sf3_2, batched=True)

In [ ]:
# Define trainer and do training
trainer_1 = Trainer(model=relation_extractor, args=training_args_1, train_dataset=train_dataset_1, eval_dataset=valid_dataset_1)
trainer_1.train()

In [ ]:
trainer_2 = Trainer(model=reconstructor, args=training_args_2, train_dataset=train_dataset_2, eval_dataset=valid_dataset_2)
trainer_2.train()

<h2> Score function 3 with GPT2

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'additional_special_tokens' : ['<ENT1>', '<ENT2>', '<RELATION>'], 'pad_token': '[PAD]'})

In [ ]:
# Define dataset and training arguement
dataset = load_from_disk('data/single-ollie-pos-only')
training_args_1_gpt2 = TrainingArguments("data/single-ollie-sf3-1-GPT2", evaluation_strategy="epoch")
training_args_2_gpt2 = TrainingArguments("data/single-ollie-sf3_2-GPT2", evaluation_strategy="epoch")

In [ ]:
# Load model
relation_extractor = GPT2LMHeadModel.from_pretrained('gpt2')
relation_extractor.resize_token_embeddings(len(tokenizer))

In [ ]:
reconstructor = GPT2LMHeadModel.from_pretrained('gpt2')
reconstructor.resize_token_embeddings(len(tokenizer))

In [ ]:
def preprocess_sf3_1_gpt2(examples):
    inputs = ['%s <ENT1> %s <ENT2> %s <RELATION> %s' % (sent, ent1, ent2, rel) for ent1, ent2, sent, rel in zip(examples['ent1'], examples['ent2'], examples['sent'], examples['rel'])]
    input_tokenized = tokenizer(inputs, padding='max_length', truncation=True, max_length=100)
    return {'input_ids': input_tokenized['input_ids'],
            'attention_mask': input_tokenized['attention_mask'],
            'labels' : input_tokenized['input_ids']}

train_dataset_1_gpt2 = dataset['train'].map(preprocess_sf3_1_gpt2, batched=True)
valid_dataset_1_gpt2 = dataset['valid'].map(preprocess_sf3_1_gpt2, batched=True)

def preprocess_sf3_2_gpt2(examples):
    inputs = ['<ENT1> %s <ENT2> %s <RELATION> %s <SENT> %s' % (ent1, ent2, rel, sent) for ent1, ent2, sent, rel in zip(examples['ent1'], examples['ent2'], examples['sent'], examples['rel'])]
    input_tokenized = tokenizer(inputs, padding='max_length', truncation=True, max_length=100)
    return {'input_ids': input_tokenized['input_ids'],
            'attention_mask': input_tokenized['attention_mask'],
            'labels' : input_tokenized['input_ids']}

train_dataset_2_gpt2 = dataset['train'].map(preprocess_sf3_2_gpt2, batched=True)
valid_dataset_2_gpt2 = dataset['valid'].map(preprocess_sf3_2_gpt2, batched=True)

In [ ]:
# Define trainer and do training
trainer_1 = Trainer(model=relation_extractor, args=training_args_1_gpt2, train_dataset=train_dataset_1_gpt2, eval_dataset=valid_dataset_1_gpt2)
trainer_1.train()

In [ ]:
trainer_2 = Trainer(model=reconstructor, args=training_args_2_gpt2, train_dataset=train_dataset_2_gpt2, eval_dataset=valid_dataset_2_gpt2)
trainer_2.train()

<h1> Testing

<h2> Score function 1

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('data/single-ollie/checkpoint-3500')
# model = ScoreFunction2.from_pretrained('data/single-ollie2')
model.eval()

In [ ]:
sparse_retriever = SparseRetrieveSentForPairCoOccur('../data/corpus/small_sent.txt', 'data/occur.json')

In [ ]:
ent1 = 'data mining'
ent2 = 'machine learning'
s = 'in this paper, we show that by using the fuzzy statistics analysis and the data mining technology, the target - oriented fuzzy correlation rules can be obtained from a given database.'
sent = sparse_retriever.retrieve(ent1, ent2)
test_list = [{'sent' : s, 'ent1' : ent1, 'ent2' : ent2, 'labels' : 1} for s in sent]
# test_list = [{'sent' : s, 'ent1' : ent1, 'ent2' : ent2, 'labels' : 1}]
print(len(test_list))

In [ ]:
temp_dict = json.load(open('data/my_dataset.json'))
test_list = temp_dict['valid'][:200]

In [ ]:
valid_df = pandas.DataFrame.from_dict(test_list)

In [ ]:
# Function that help generate score
def get_score(sents:List[str], ent1s:List[str], ent2s:List[str]):
    query = ['%s <RELATION> %s' % (ent1, ent2) for ent1, ent2 in zip(ent1s, ent2s)]
    with torch.no_grad():
        inputs = BatchEncoding(tokenizer(query, sents, padding=True, truncation=True, max_length=80, return_tensors='pt'))
        output = model(**inputs)
        s = Softmax(1)
        return s(output.logits)

In [ ]:
# Get logits score
val_output = get_score(valid_df.sent.to_list(), valid_df.ent1.to_list(), valid_df.ent2.to_list())
# Get prediction label
cls_result = np.argmax(val_output.numpy(), axis=1)
# Get prediction score
cls_score = val_output.numpy()[:, 1]
# Get ground truth
val_label = np.array(valid_df.labels.to_list())
# Get correct ones
correct_prediction = val_label == cls_result
# Sum the number of correct ones
correct_num = np.sum(correct_prediction)
# Get the wrong prediction idx
wrong_prediction_idx = np.arange(0, len(val_label))[val_label != cls_result]
# Get the wrong ones
wrong_samples = [(cls_result[idx], valid_df.labels[idx], valid_df.ent1[idx], valid_df.ent2[idx], valid_df.sent[idx]) for idx in wrong_prediction_idx]
# Write the wrong ones to file
with open('data/wrong_prediction.tsv', 'w') as f_out:
    w = csv.writer(f_out, delimiter='\t')
    w.writerows(wrong_samples)

# Get rank
rank_ids = ntopidx(len(cls_score), cls_score)
rank_list = [(cls_score[idx], valid_df.ent1[idx], valid_df.ent2[idx], valid_df.sent[idx]) for idx in rank_ids]
with open('data/rank_list.tsv', 'w') as f_out:
    w = csv.writer(f_out, delimiter='\t')
    w.writerows(rank_list)

<h2> Score function 3

In [ ]:
from datasets import Dataset
import pandas as pd

In [ ]:
relation_extractor = BartForConditionalGeneration.from_pretrained('data/single-ollie-sf3_1/checkpoint-6500')
relation_extractor.eval()

In [7]:
sparse_retriever = SparseRetrieveSentForPairCoOccur('../data/corpus/small_sent.txt', 'data/occur.json')

In [9]:
ent1 = 'data mining'
ent2 = 'machine learning'
s = 'in this paper, we show that by using the fuzzy statistics analysis and the data mining technology, the target - oriented fuzzy correlation rules can be obtained from a given database.'
sent = sparse_retriever.retrieve(ent1, ent2)
test_list = [{'sent' : s, 'ent1' : ent1, 'ent2' : ent2, 'labels' : 1} for s in sent]
# test_list = [{'sent' : s, 'ent1' : ent1, 'ent2' : ent2, 'labels' : 1}]
print(len(test_list))

191


In [24]:
test_df = pd.DataFrame(test_list)
test_dataset = Dataset.from_pandas(test_df)
test_dataset_input = preprocess_sf3_1(test_dataset)
output = relation_extractor.generate(torch.LongTensor(test_dataset_input['input_ids']))
test_df['extraction'] = tokenizer.batch_decode(output)
test_df[['ent1', 'ent2', 'extraction', 'sent']].to_csv('test_result.tsv', sep='\t', index=False)